## Importing the necesssary libraries and loading the dataset:
---

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import warnings
warnings.filterwarnings('ignore')

#   Initialize the Spark session
spark = SparkSession.builder \
    .master("local") \
    .appName("Exercise3") \
    .getOrCreate()

df = spark.read.format('org.apache.spark.sql.json') \
        .load("icc_results.json")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/24 14:57:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Finding number of test and odi played each year :
---

In [54]:
matches = df.withColumn("mode", col("scheduleentry.description"))\
                    .withColumn("year", col("scheduleentry.matchDate"))\
                    .select('mode','year').dropna()



matches = matches.withColumn('Mode', split(matches['mode'], ' ').getItem(1)).dropna()\
                 .withColumn('Year', year(matches.year)).dropna()

matches = matches.filter((matches.Mode == 'Test') | (matches.Mode == 'ODI')) #filtering t20I

matches = matches.groupBy("Year","Mode").count()\
                 .orderBy(['Mode'],ascending=False)

matches.show()


+----+----+-----+
|Year|Mode|count|
+----+----+-----+
|2020|Test|   27|
|2019|Test|   39|
|2014|Test|   41|
|2016|Test|   47|
|2017|Test|   49|
|2013|Test|   37|
|2018|Test|   48|
|2012|Test|    9|
|2015|Test|   45|
|2014| ODI|   88|
|2020| ODI|   56|
|2015| ODI|   89|
|2012| ODI|    1|
|2019| ODI|   83|
|2013| ODI|   94|
|2017| ODI|  108|
|2018| ODI|   78|
|2016| ODI|   80|
+----+----+-----+



In [47]:
distinct_years = matches.drop_duplicates(subset = ['Year']).drop('Mode','count').orderBy(['Year'],ascending=False)
distinct_years.show()

+----+
|Year|
+----+
|2020|
|2019|
|2018|
|2017|
|2016|
|2015|
|2014|
|2013|
|2012|
+----+



In [59]:
finaldf = distinct_years.withColumn('TESTS',matches("count").collect(10)).show()

TypeError: 'DataFrame' object is not callable